In [4]:
from sqlalchemy import Table, Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [2]:
# DB接続
engine = create_engine('sqlite:///test.db')

Base = declarative_base()

In [3]:
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer)
    id.primary_key=True
    name = Column(String)
    age = Column(Integer)
    
    # Postとのリレーション
    posts = relationship('Post', backref='users')

In [5]:
class Post(Base):
    __tablename__ = 'posts'
    id = Column(Integer)
    id.primary_key = True
    users_id = Column(Integer, ForeignKey('users.id'))
    title = Column(String)
    body = Column(Integer)

    # Userとのリレーション
    user = relationship('User')


In [6]:
Base.metadata.create_all(engine)

In [7]:
# セッションの生成
Session = sessionmaker(bind=engine)
session = Session()

In [8]:
# サンプルデータ挿入
session.add(User(id=1, name="Suzuki", age=19))
session.add(User(id=2, name="Tanaka", age=21))
session.add(User(id=3, name="Sato", age=21))

session.add(Post(users_id=1, title="朝の体操", body="ラジオ体操で元気いっぱい"))
session.add(Post(users_id=1, title="今日の夕食", body="カレーラスがとても美味しかった。"))
session.add(Post(users_id=2, title="仕事", body="今日はDjangoでAPI作成。"))
session.add(Post(users_id=2, title="Python楽しい", body="Python楽しいですよね！！"))
session.commit()


C:\Users\naoki\AppData\Local\Temp\ipykernel_17276\8331238.py:2: SAWarning: relationship 'Post.user' will copy column users.id to column posts.users_id, which conflicts with relationship(s): 'Post.users' (copies users.id to posts.users_id), 'User.posts' (copies users.id to posts.users_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="posts,users"' to the 'Post.user' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
  session.add(User(id=1, name="Suzuki", age=19))


In [9]:
# inner joinのサンプル
users = session.query(User).join(Post, User.id == Post.users_id).all()

In [10]:
for user in users:
    print(f'{user.name}さんの投稿')
    for post in user.posts:
        print(f"|- タイトル: {post.title}")
    print()

Suzukiさんの投稿
|- タイトル: 朝の体操
|- タイトル: 今日の夕食
|- タイトル: 朝の体操
|- タイトル: 今日の夕食

Tanakaさんの投稿
|- タイトル: 仕事
|- タイトル: Python楽しい
|- タイトル: 仕事
|- タイトル: Python楽しい



In [11]:
import pprint
# inner joinのサンプル
users = session.query(User).join(Post, User.id == Post.users_id).all()

user = users[0]

pprint.pprint(vars(user))


{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001B12038F850>,
 'age': 19,
 'id': 1,
 'name': 'Suzuki',
 'posts': [<__main__.Post object at 0x000001B12038CC10>,
           <__main__.Post object at 0x000001B12038DF60>]}
